In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 6.3 MB/s eta 0:00:00


### 1. 필요한 라이브러리 가져오기

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchmetrics import Accuracy
from torchmetrics.aggregation import MeanMetric

### 2. Build Config

In [8]:
title = 'FashionMNIST'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_root ='data'
epochs = 5
batch_size = 64
base_lr = 0.01
momentum = 0.9
checkpoint_dir = 'checkpoint'

In [9]:
# Build directory
os.makedirs(checkpoint_dir, exist_ok=True)

### 3. Build Datasets

In [13]:
train_transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, ), (0.5, ))
])

train_data = FashionMNIST(data_root, train=True, download=True, transform=train_transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, ), (0.5, ))
])

val_data = FashionMNIST(data_root, train=False, download=True, transform=val_transform)
val_loader = DataLoader(val_data, batch_size=batch_size)

100%|██████████| 26421880/26421880 [00:01<00:00, 13943814.88it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 273510.94it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4971451.42it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13832336.32it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### 4. Build Model

In [27]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.layers = nn.ModuleList()
    self.layers.append(nn.Sequential(
        nn.Linear(28 * 28, 500),
        nn.ReLU()
    ))
    self.layers.append(nn.Sequential(
        nn.Linear(500, 500),
        nn.ReLU()
    ))
    self.head = nn.Linear(500, 10)
  def forward(self, x):
    x = x.reshape((x.shape[0], -1))
    for layer in self.layers:
      x = layer(x)
    x = self.head(x)
    return x

In [28]:
model = MyModel()
print(model)

MyModel(
  (layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=784, out_features=500, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=500, out_features=500, bias=True)
      (1): ReLU()
    )
  )
  (head): Linear(in_features=500, out_features=10, bias=True)
)


### 5. Build Optimizer, Scheduler, loss function, metric function

In [29]:
optimizer = optim.SGD(model.parameters(), lr = base_lr, momentum = momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs * len(train_loader))
loss_fn = nn.CrossEntropyLoss()

metric_fn = Accuracy(task='multiclass', num_classes=10)
metric_fn = metric_fn.to(device)

### 6. Define Train Loop

In [30]:
def train(loader, model, optimizer, scheduler, loss_fn, metric_fn, device):
  model.train()

  loss_mean = MeanMetric()
  metric_mean = MeanMetric()

  for inputs, targets in loader:
    inputs = inputs.to(device)
    targets = targets.to(device)

    # Forward
    outputs = model(inputs)
    loss = loss_fn(outputs, targets)
    metric = metric_fn(outputs, targets)

    #Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #Update
    loss_mean.update(loss.to('cpu'))
    metric_mean.update(metric.to('cpu'))

    scheduler.step()

      # Summarize statistics
  summary = {'loss': loss_mean.compute(), 'metric': metric_mean.compute()}

  return summary

### 7. Define Evaluate Loop

In [31]:
def evaluate(loader, model, loss_fn, metric_fn, device):
    # Set model to evaluation mode
    model.eval()

    # Create average meters to measure loss and accuracy
    loss_mean = MeanMetric()
    metric_mean = MeanMetric()

    # Evalute model for one epoch
    for inputs, targets in loader:
        # Move data to device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward
        with torch.no_grad():
            outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        metric = metric_fn(outputs, targets)

        # Update statistics
        loss_mean.update(loss.to('cpu'))
        metric_mean.update(metric.to('cpu'))

    # Summarize statistics
    summary = {'loss': loss_mean.compute(), 'metric': metric_mean.compute()}

    return summary

### 8. Define Main Loop

In [32]:
# Main loop
for epoch in range(epochs):
    # train one epoch
    train_summary = train(train_loader, model, optimizer, scheduler, loss_fn, metric_fn, device)

    # evaluate one epoch
    val_summary = evaluate(val_loader, model, loss_fn, metric_fn, device)

    # print log
    print((f'Epoch {epoch+1}: '
           + f'Train Loss {train_summary["loss"]:.04f}, '
           + f'Train Accuracy {train_summary["metric"]:.04f}, '
           + f'Test Loss {val_summary["loss"]:.04f}, '
           + f'Test Accuracy {val_summary["metric"]:.04f}'))

    # save model
    state_dict = {
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    checkpoint_path = f'{checkpoint_dir}/{title}_last.pth'
    torch.save(state_dict, checkpoint_path)

Epoch 1: Train Loss 0.5490, Train Accuracy 0.8003, Test Loss 0.4235, Test Accuracy 0.8487
Epoch 2: Train Loss 0.3742, Train Accuracy 0.8635, Test Loss 0.3751, Test Accuracy 0.8657
Epoch 3: Train Loss 0.3246, Train Accuracy 0.8822, Test Loss 0.3642, Test Accuracy 0.8666
Epoch 4: Train Loss 0.2928, Train Accuracy 0.8930, Test Loss 0.3425, Test Accuracy 0.8777
Epoch 5: Train Loss 0.2747, Train Accuracy 0.8999, Test Loss 0.3378, Test Accuracy 0.8807
